In [2]:
pretrain_dir = None # model dir for resuming training. if None, train from scrach

one_fold = False # if True, train model at only first fold. use if you try a new idea quickly.
run_test = False # if True, use small data. you can check whether this code run or not
denoise = True # if True, use train data whose signal_to_noise > 1

ae_epochs = 10 # epoch of training of denoising auto encoder
ae_epochs_each = 5 # epoch of training of denoising auto encoder each time. 
                   # I use train data (seqlen = 107) and private test data (seqlen = 130) for auto encoder training.
                   # I dont know how to easily fit keras model to use both of different shape data simultaneously, 
                   # so I call fit function several times. 
ae_batch_size = 32

# epochs_list = [30, 10] # 3, 3, 5, 5]
epochs_list = [10]
batch_size_list = [8, 16] #32, 64, 128, 256] 

## copy pretrain model to working dir
import shutil
import glob
if pretrain_dir is not None:
    for d in glob.glob(pretrain_dir + "*"):
        shutil.copy(d, ".")
    
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import gc
import os
import matplotlib.pyplot as plt
%matplotlib inline

### Load

In [4]:
import json
import glob
from tqdm import tqdm

train = pd.read_json("datasets/input/train.json", lines=True)
if denoise:
    train = train[train.signal_to_noise > 1].reset_index(drop=True)

test = pd.read_json("datasets/input/test.json", lines=True)
test_pub = test[test["seq_length"] == 107]
test_pri = test[test["seq_length"] == 130]
# sub = pd.read_csv("datasets/input/sample_submission.csv")
# sub = sub.iloc[:3]

if run_test:  ## to test
    train = train[:30]
    test_pub = test_pub[:30]
    test_pri = test_pri[:30]

In [5]:
bpps = []
for id in tqdm(train["id"]):
    a = np.load(f"datasets/input/bpps/{id}.npy")
    bpps.append(a)

bpps = np.array(bpps)

bpps_pub = []
for id in tqdm(test_pub["id"]):
    a = np.load(f"datasets/input/bpps/{id}.npy")
    bpps_pub.append(a)

bpps_pub = np.array(bpps_pub)

bpps_pri = []
for id in tqdm(test_pri["id"]):
    a = np.load(f"datasets/input/bpps/{id}.npy")
    bpps_pri.append(a)

bpps_pri = np.array(bpps_pri)

100%|██████████| 3005/3005 [00:02<00:00, 1491.33it/s]


In [6]:
bpps.shape

(2096, 107, 107)

In [5]:
print(train.shape)
train.head(2)

(2096, 19)


,index,id,sequence,structure,predicted_loop_type,signal_to_noise,SN_filter,seq_length,seq_scored,reactivity_error,deg_error_Mg_pH10,deg_error_pH10,deg_error_Mg_50C,deg_error_50C,reactivity,deg_Mg_pH10,deg_pH10,deg_Mg_50C,deg_50C
0,0,id_001f94081,GGAAAAGCUCUAAUAACAGGAGACUAGGACUACGUAUUUCUAGGUA...,.....((((((.......)))).)).((.....((..((((((......,EEEEESSSSSSHHHHHHHSSSSBSSXSSIIIIISSIISSSSSSHHH...,6.894,1,107,68,"[0.1359, 0.20700000000000002, 0.1633, 0.1452, ...","[0.26130000000000003, 0.38420000000000004, 0.1...","[0.2631, 0.28600000000000003, 0.0964, 0.1574, ...","[0.1501, 0.275, 0.0947, 0.18660000000000002, 0...","[0.2167, 0.34750000000000003, 0.188, 0.2124, 0...","[0.3297, 1.5693000000000001, 1.1227, 0.8686, 0...","[0.7556, 2.983, 0.2526, 1.3789, 0.637600000000...","[2.3375, 3.5060000000000002, 0.3008, 1.0108, 0...","[0.35810000000000003, 2.9683, 0.2589, 1.4552, ...","[0.6382, 3.4773, 0.9988, 1.3228, 0.78770000000..."
1,2,id_006f36f57,GGAAAGUGCUCAGAUAAGCUAAGCUCGAAUAGCAAUCGAAUAGAAU...,.....((((.((.....((((.(((.....)))..((((......)...,EEEEESSSSISSIIIIISSSSMSSSHHHHHSSSMMSSSSHHHHHHS...,8.800,1,107,68,"[0.0931, 0.13290000000000002, 0.11280000000000...","[0.1365, 0.2237, 0.1812, 0.1333, 0.1148, 0.160...","[0.17020000000000002, 0.178, 0.111, 0.091, 0.0...","[0.1033, 0.1464, 0.1126, 0.09620000000000001, ...","[0.14980000000000002, 0.1761, 0.1517, 0.116700...","[0.44820000000000004, 1.4822, 1.1819, 0.743400...","[0.2504, 1.4021, 0.9804, 0.49670000000000003, ...","[2.243, 2.9361, 1.0553, 0.721, 0.6396000000000...","[0.5163, 1.6823000000000001, 1.0426, 0.7902, 0...","[0.9501000000000001, 1.7974999999999999, 1.499..."


In [6]:
print(test.shape)
test.head(2)

(3634, 7)


,index,id,sequence,structure,predicted_loop_type,seq_length,seq_scored
0,0,id_00073f8be,GGAAAAGUACGACUUGAGUACGGAAAACGUACCAACUCGAUUAAAA...,......((((((((((.(((((.....))))))))((((((((......,EEEEEESSSSSSSSSSBSSSSSHHHHHSSSSSSSSSSSSSSSSHHH...,107,68
1,1,id_000ae4237,GGAAACGGGUUCCGCGGAUUGCUGCUAAUAAGAGUAAUCUCUAAAU...,.....((((..((((((...(((((.....((((....)))).......,EEEEESSSSIISSSSSSIIISSSSSIIIIISSSSHHHHSSSSIIII...,130,91


### Target

In [7]:
targets = ["reactivity", "deg_Mg_pH10", "deg_pH10", "deg_Mg_50C", "deg_50C"]

y_train = []
seq_len = train["seq_length"].iloc[0]
seq_len_target = train["seq_scored"].iloc[0]
ignore = -10000
ignore_length = seq_len - seq_len_target

for target in targets:
    y = np.vstack(train[target])
    dummy = np.zeros([y.shape[0], ignore_length]) + ignore
    y = np.hstack([y, dummy])
    y_train.append(y)

y = np.stack(y_train, axis=2)
y.shape

(2096, 107, 5)

### Structure adjacent

In [8]:
def get_structure_adj(train):
    """Get adjacency matrix from structure sequence.
    It calculate adjacent matrix of each base pair
    but eventually ignore difference of base pair
    and intergrate into one matrix"""

    adj_structures = []
    for seq_index in tqdm(range(len(train))):
        seq_length = train["seq_length"].iloc[seq_index]
        structure = train["structure"].iloc[seq_index]
        sequence = train["sequence"].iloc[seq_index]

        struc_stack = []
        adj_structure_dict = {
            ("A", "U"): np.zeros([seq_length, seq_length]),
            ("C", "G"): np.zeros([seq_length, seq_length]),
            ("U", "G"): np.zeros([seq_length, seq_length]),
            ("U", "A"): np.zeros([seq_length, seq_length]),
            ("G", "C"): np.zeros([seq_length, seq_length]),
            ("G", "U"): np.zeros([seq_length, seq_length]),
        }

        # a_structure = np.zeros([seq_length, seq_length])
        for i in range(seq_length):
            if structure[i] == "(":
                struc_stack.append(i)
            elif structure[i] == ")":
                start = struc_stack.pop()

                # a_structure[start, i] = 1
                # a_structure[i, start] = 1
                adj_structure_dict[(sequence[start], sequence[i])][start, i] = 1
                adj_structure_dict[(sequence[i], sequence[start])][i, start] = 1

        adj_structure = np.stack([i for i in adj_structure_dict.values()], axis=2)
        adj_structure = np.sum(adj_structure, axis=2, keepdims=True)
        adj_structures.append(adj_structure)

    adj_structures = np.array(adj_structures)
    print(adj_structures.shape)
    return adj_structures

In [9]:
adj_struc = get_structure_adj(train)
adj_struc_pub = get_structure_adj(test_pub)
adj_struc_pri = get_structure_adj(test_pri)

100%|██████████| 2096/2096 [00:00<00:00, 4481.22it/s]


(2096, 107, 107, 1)


100%|██████████| 629/629 [00:00<00:00, 4352.84it/s]


(629, 107, 107, 1)


100%|██████████| 3005/3005 [00:01<00:00, 2911.53it/s]


(3005, 130, 130, 1)


### Distance adjacent

In [10]:
def get_distance_matrix(bpps):
    """Adjacent matrix based on distance on the sequence.
    D[i, j] = 1 / (abs(i - j) + 1) ** pow, pow = 1, 2, 4"""
    idx = np.arange(bpps.shape[1])

    distance_seq = [np.abs(idx[i] - idx) for i in range(len(idx))]
    distance_seq = np.array(distance_seq) + 1
    distance_seq = 1 / distance_seq

    distance_seq = distance_seq[None, :, :]
    distance_seq = np.repeat(distance_seq, bpps.shape[0], axis=0)

    distance_powered = [distance_seq**pow for pow in [1, 2, 4]]
    distance_seq = np.stack(distance_powered, axis=3)

    print(distance_seq.shape)
    return distance_seq

In [11]:
distance_matrix = get_distance_matrix(bpps)
distance_matrix_pub = get_distance_matrix(bpps_pub)
distance_matrix_pri = get_distance_matrix(bpps_pri)

(2096, 107, 107, 3)
(629, 107, 107, 3)
(3005, 130, 130, 3)


In [12]:
# concat adjacent
adjacents = np.concatenate(
    [bpps[:, :, :, None], adj_struc, distance_matrix], axis=3
).astype(np.float32)
del bpps, adj_struc, distance_matrix

adjacents_pub = np.concatenate(
    [bpps_pub[:, :, :, None], adj_struc_pub, distance_matrix_pub], axis=3
).astype(np.float32)
del bpps_pub, adj_struc_pub, distance_matrix_pub

adjacents_pri = np.concatenate(
    [bpps_pri[:, :, :, None], adj_struc_pri, distance_matrix_pri], axis=3
).astype(np.float32)
del bpps_pri, adj_struc_pri, distance_matrix_pri

adjacents.shape, adjacents_pub.shape, adjacents_pri.shape

((2096, 107, 107, 5), (629, 107, 107, 5), (3005, 130, 130, 5))

### Node

In [13]:
def one_hot_encoder(vocab):
    """One hot encoder"""
    mapping = {}
    n = len(vocab)
    for i, s in enumerate(vocab):
        mapping[s] = [0] * n
        mapping[s][i] = 1
    return mapping


def get_input(train):
    """Get node features, which is one hot encoded"""
    vocab = ["A", "U", "G", "C"]
    encoder = one_hot_encoder(vocab)
    X_node = np.stack(
        train["sequence"].apply(
            lambda sequence: list(map(lambda base: encoder[base], sequence))
        )
    )

    
    vocab = ["S", "M", "I", "B", "H", "E", "X"]
    encoder = one_hot_encoder(vocab)
    X_loop = np.stack(
        train["predicted_loop_type"].apply(
            lambda string: list(map(lambda char: encoder[char], string))
        )
    )

   
    # vocab = [".", "(", ")"]
    # encoder = one_hot_encoder(vocab)
    # X_structure = np.stack(
    #     train["structure"].apply(
    #         lambda structure: list(map(lambda base: encoder[base], structure))
    #     )
    # )

    X_node = np.concatenate([X_node, X_loop], axis=2)

    # interaction
    a = np.sum(
        X_node * (2 ** np.arange(X_node.shape[2])[None, None, :]),
        axis=2,
    )

    vocab = sorted(set(a.flatten()))
    print(vocab)

    ohes = []
    for v in vocab:
        ohes.append(a == v)

    ohes = np.stack(ohes, axis=2)
    X_node = np.concatenate([X_node, ohes], axis=2).astype(np.float32)
    print(X_node.shape)
    return X_node

In [14]:
X_node = get_input(train)
X_node_pub = get_input(test_pub)
X_node_pri = get_input(test_pri)

[17, 18, 20, 24, 33, 34, 36, 40, 65, 66, 68, 72, 129, 130, 132, 136, 257, 258, 260, 264, 513, 514, 516, 520, 1025, 1026, 1028, 1032]
(2096, 107, 39)
[17, 18, 20, 24, 33, 34, 36, 40, 65, 66, 68, 72, 129, 130, 132, 136, 257, 258, 260, 264, 513, 514, 516, 520, 1025, 1026, 1028, 1032]
(629, 107, 39)
[17, 18, 20, 24, 33, 34, 36, 40, 65, 66, 68, 72, 129, 130, 132, 136, 257, 258, 260, 264, 513, 514, 516, 520, 1025, 1026, 1028, 1032]
(3005, 130, 39)


### Model

In [15]:
import tensorflow as tf
from tensorflow import keras
from keras import layers, backend, Model, Input, optimizers
from keras.layers import (
    Conv1D,
    Conv2D,
    Permute,
    Lambda,
    Concatenate,
    Dense,
    Add,
    LayerNormalization,
    Dropout,
    LeakyReLU,
    SpatialDropout1D
)

2023-12-15 16:26:50.450401: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-15 16:26:50.642189: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-15 16:26:50.642224: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-15 16:26:50.675194: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-15 16:26:50.744729: I tensorflow/core/platform/cpu_feature_guar

In [16]:
def mcrmse(y_test, y_pred, seq_len_target=seq_len_target):
    """Calculate mcrmse score by using numpy"""
    y_test = y_test[:, :seq_len_target]
    y_pred = y_pred[:, :seq_len_target]

    score = np.mean(
        np.sqrt(
            np.mean(
                np.mean((y_pred - y_test) ** 2, axis=1),
                axis=0,
            )
        )
    )
    return score


def mcrmse_loss(y_test, y_pred, seq_len_target=seq_len_target):
    """Calculate mcrmse loss by using tf"""
    y_test = y_test[:, :seq_len_target]
    y_pred = y_pred[:, :seq_len_target]

    loss = tf.reduce_mean(
        tf.sqrt(
            tf.reduce_mean(
                tf.reduce_mean((y_test - y_pred) ** 2, axis=1),
                axis=0,
            )
        )
    )
    return loss


def attention(x_inner, x_outer, n_factor, dropout):
    x_Q = Conv1D(
        n_factor,
        1,
        activation="linear",
        kernel_initializer="glorot_uniform",
        bias_initializer="glorot_uniform",
    )(x_inner)

    x_K = Conv1D(
        n_factor,
        1,
        activation="linear",
        kernel_initializer="glorot_uniform",
        bias_initializer="glorot_uniform",
    )(x_outer)

    x_V = Conv1D(
        n_factor,
        1,
        activation="linear",
        kernel_initializer="glorot_uniform",
        bias_initializer="glorot_uniform",
    )(x_outer)

    x_KT = Permute((2, 1))(x_K)
    res = Lambda(lambda c: backend.batch_dot(c[0], c[1]) / np.sqrt(n_factor))(
        [x_Q, x_KT]
    )

    # res = tf.expand_dims(res, axis=3)
    # res = Conv2D(16, 3, 1, padding="same", activation="relu")(res)
    # res = Conv2D(1, 3, 1, padding="same", activation="relu")(res)
    # res = tf.squeeze(res, axis=3)

    att = Lambda(lambda c: backend.softmax(c, axis=-1))(res)
    att = Lambda(lambda c: backend.batch_dot(c[0], c[1]))([att, x_V])
    return att


def multi_head_attention(x, y, n_factor, n_head, dropout):
    if n_head == 1:
        att = attention(x, y, n_factor, dropout)
    else:
        n_factor_head = n_factor // n_head
        heads = [attention(x, y, n_factor_head, dropout) for i in range(n_head)]
        att = Concatenate()(heads)
        att = Dense(
            n_factor,
            kernel_initializer="glorot_uniform",
            bias_initializer="glorot_uniform",
        )(att)

    x = Add()([x, att])
    x = LayerNormalization()(x)
    if dropout > 0:
        x = Dropout(dropout)(x)

    return x


def res(x, unit, kernel=3, rate=0.1):
    h = Conv1D(unit, kernel, 1, padding="same", activation=None)(x)
    h = LayerNormalization()(h)
    h = LeakyReLU()(h)
    h = Dropout(rate)(h)
    return Add()([x, h])


def forward(x, unit, kernel=3, rate=0.1):
    # h = Dense(unit, None)(x)
    h = Conv1D(unit, kernel, 1, padding="same", activation=None)(x)
    h = LayerNormalization()(h)
    h = Dropout(rate)(h)

    # h = tf.keras.activations.swish(h)
    h = LeakyReLU()(h)
    h = res(h, unit, kernel, rate)
    return h


def adj_attention(x, adj, unit, n=2, rate=0.1):
    x_a = x
    x_as = []
    for i in range(n):
        x_a = forward(x_a, unit)
        x_a = tf.matmul(adj, x_a)  # aggregate neighborhoods
        x_as.append(x_a)

    if n == 1:
        x_a = x_as[0]
    else:
        x_a = Concatenate()(x_as)

    x_a = forward(x_a, unit)
    return x_a


def get_optimizer(optim="Adam"):
    if optim == "Adam":
        optimizer = optimizers.Adam()
    elif optim == "SGD":
        optimizer = optimizers.SGD(0.05, momentum=0.9, nesterov=True)
    return optimizer


def get_base(config):
    """Base model architecture.
    node, adj -> middle feature"""
    node = Input(shape=(None, X_node.shape[2]), name="node")
    adj = Input(shape=(None, None, adjacents.shape[3]), name="adj")

    adj_learned = Dense(1, "relu")(adj)
    adj_all = Concatenate(axis=3)([adj, adj_learned])

    xs = []
    xs.append(node)
    x1 = forward(node, 128, kernel=3, rate=0.0)
    x2 = forward(x1, 64, kernel=6, rate=0.0)
    x3 = forward(x2, 32, kernel=15, rate=0.0)
    x4 = forward(x3, 16, kernel=30, rate=0.0)
    x = Concatenate()([x1, x2, x3, x4])

    for unit in [64, 32]:
        x_as = []
        for i in range(adj_all.shape[3]):
            x_a = adj_attention(x, adj_all[:, :, :, i], unit, rate=0.0)
            x_as.append(x_a)

        x_c = forward(x, unit, kernel=30)

        x = Concatenate()(x_as + [x_c])
        x = forward(x, unit)
        x = multi_head_attention(x, x, unit, 4, 0.0)
        xs.append(x)

    x = Concatenate()(xs)
    model = Model(inputs=[node, adj], outputs=[x])
    return model


def get_ae_model(base, config):
    """Denoising auto encoder part.\n
    node, adj -> middle feature -> node"""
    node = Input(shape=(None, X_node.shape[2]), name="node")
    adj = Input(shape=(None, None, adjacents.shape[3]), name="adj")

    x = base([SpatialDropout1D(0.3)(node), adj])
    x = forward(x, 64, rate=0.3)
    p = Dense(X_node.shape[2], "sigmoid")(x)

    loss = -tf.reduce_mean(
        20 * node * tf.math.log(p + 1e-4) + (1 - node) * tf.math.log(1 - p + 1e-4)
    )
    model = Model(inputs=[node, adj], outputs=[loss])

    optim = get_optimizer("Adam")
    model.compile(optimizer=optim, loss=lambda t, y: y)
    return model


def get_model(base, config):
    """Regression part\n
    node, adj -> middle feature -> prediction of targets"""
    node = Input(shape=(None, X_node.shape[2]), name="node")
    adj = Input(shape=(None, None, adjacents.shape[3]), name="adj")

    x = base([node, adj])
    x = forward(x, 128, rate=0.4)
    x = Dense(5, None)(x)

    model = Model(inputs=[node, adj], outputs=[x])

    optim = get_optimizer()
    model.compile(optimizer=optim, loss=mcrmse_loss)
    return model

### Pretrain

In [18]:
# Train denoising auto encoder model using all data
config = {}

if ae_epochs > 0:
    base = get_base(config)
    ae_model = get_ae_model(base, config)

    # TODO: simultaneous train
    for i in range(ae_epochs // ae_epochs_each):
        print(f"-----{i}-----")
        print("--- Train ---")
        ae_model.fit(
            [X_node, adjacents],
            [X_node[:, 0]],
            epochs=ae_epochs_each,
            batch_size=ae_batch_size,
        )

        print("--- Public ---")
        ae_model.fit(
            [X_node_pub, adjacents_pub],
            [X_node_pub[:, 0]],
            epochs=ae_epochs_each,
            batch_size=ae_batch_size,
        )

        print("--- Private ---")
        ae_model.fit(
            [X_node_pri, adjacents_pri],
            [X_node_pri[:, 0]],
            epochs=ae_epochs_each,
            batch_size=ae_batch_size,
        )

        gc.collect()

    print("***** Save AE model *****")
    base.save_weights("./base_ae")

-----0-----
--- Train ---
Epoch 1/5
66/66 [==============================] - 42s 112ms/step - loss: 0.9235
Epoch 2/5
66/66 [==============================] - 7s 107ms/step - loss: 0.3501
Epoch 3/5
66/66 [==============================] - 7s 107ms/step - loss: 0.1807
Epoch 4/5
66/66 [==============================] - 7s 108ms/step - loss: 0.1178
Epoch 5/5
66/66 [==============================] - 7s 108ms/step - loss: 0.0880
--- Public ---
Epoch 1/5
20/20 [==============================] - 6s 332ms/step - loss: 0.0725
Epoch 2/5
20/20 [==============================] - 2s 107ms/step - loss: 0.0730
Epoch 3/5
20/20 [==============================] - 2s 107ms/step - loss: 0.0667
Epoch 4/5
20/20 [==============================] - 2s 107ms/step - loss: 0.0612
Epoch 5/5
20/20 [==============================] - 2s 107ms/step - loss: 0.0562
--- Private ---


: 

In [18]:
!python3 -c "import tensorflow as tf; print(tf.config.list_physical_devices('GPU'))"

2023-12-15 16:13:23.050812: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-15 16:13:23.074401: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-15 16:13:23.074448: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-15 16:13:23.075127: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-15 16:13:23.079473: I tensorflow/core/platform/cpu_feature_guar